In [1]:
from cycada_utils import *

Figure directory exists.


In [2]:
n_epochs     = 100
batch_size   = 1
lr           = 0.0002
b1           = 0.5 
b2           = 0.999 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'On {device}')
root = '/hpc/home/srs108/gan_practice/data/Places365/train/'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

On cuda:0


In [3]:
class MultiCityDataset(Dataset):
    def __init__(self, root_dir, training_cities, crop_dim=0):

        '''
        training_cities = list of cities we want to train on
        testing_city = list of city to test
        '''
        # Pass in parameters
        self.crop_dim = crop_dim
        self.root_dir = root_dir
        self.training_cities = training_cities

        
        self.L =[]
        self.D = []
        self.K = []
        self.H = []
        self.M = []
        self.J = []
        self.Lu = []
        
        for city in self.training_cities:
            print('Loading: '+city)
            with open(root_dir+'/'+city+'.pkl', "rb") as fp:

                if city == 'Lahore':
                    images=pkl.load(fp)
                    for data_point in tqdm(images):
                        self.L.append(tuple((data_point['Image'][:,:,:3], data_point['PM'])))
                if city == 'Kolkata':
                    images=pkl.load(fp)
                    for data_point in tqdm(images):
                        self.K.append(tuple((data_point['Image'][:,:,:3], data_point['PM'])))
                if city == 'Dhaka':
                    images=pkl.load(fp)
                    for data_point in tqdm(images):
                        self.D.append(tuple((data_point['Image'][:,:,:3], data_point['PM'])))
                        
                if city == 'Hyderabad':
                    images = pkl.load(fp)
                    for station in tqdm(images):
                        for data_point in station:
                            self.H.append(tuple((data_point['Image'][:,:,:3], data_point['PM'])))
                if city == 'Mumbai':
                    images = pkl.load(fp)
                    for station in tqdm(images):
                        for data_point in station:
                            self.M.append(tuple((data_point['Image'][:,:,:3], data_point['PM'])))
                            
                if city == 'Jaipar':
                    images = pkl.load(fp)
                    for station in tqdm(images):
                        for data_point in station:
                            self.J.append(tuple((data_point['Image'][:,:,:3], data_point['PM'])))    
                if city == 'Lucknow':
                    images = pkl.load(fp) 
                    for station in tqdm(images):
                        for data_point in station:
                            self.Lu.append(tuple((data_point['Image'][:,:,:3], data_point['PM']))) 


        self.L_extend = random.choices(self.L, k = len(self.L) +len(self.D) +len(self.K) +len(self.H) +len(self.M) + len(self.J)+len(self.Lu))
        self.D_extend = random.choices(self.D, k = len(self.L) +len(self.D) +len(self.K) +len(self.H) +len(self.M) + len(self.J)+len(self.Lu))
        self.K_extend = random.choices(self.K, k = len(self.L) +len(self.D) +len(self.K) +len(self.H) +len(self.M) + len(self.J)+len(self.Lu))
        self.H_extend = random.choices(self.H, k = len(self.L) +len(self.D) +len(self.K) +len(self.H) +len(self.M) + len(self.J)+len(self.Lu))
        self.M_extend = random.choices(self.M, k = len(self.L) +len(self.D) +len(self.K) +len(self.H) +len(self.M) + len(self.J) +len(self.Lu))
        self.J_extend = random.choices(self.J, k = len(self.L) +len(self.D) +len(self.K) +len(self.H) +len(self.M) + len(self.J) +len(self.Lu))
        self.Lu_extend = random.choices(self.Lu, k = len(self.L) +len(self.D) +len(self.K) +len(self.H)+len(self.M) + len(self.J) +len(self.Lu))
        

    def __len__(self): return (len(self.L) +
                               len(self.D) +
                               len(self.K) +
                               len(self.H) +
                               len(self.M) +
                               len(self.J) +
                               len(self.Lu)
                                )
        
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        
        
        self.Limg = self.L_extend[idx][0]
        self.Dimg = self.D_extend[idx][0]
        self.Kimg = self.K_extend[idx][0]
        self.Himg = self.H_extend[idx][0]
        self.Mimg = self.M_extend[idx][0]
        self.Jimg = self.J_extend[idx][0]
        self.Luimg = self.Lu_extend[idx][0]


        lpm = self.L_extend[idx][1]
        dpm = self.D_extend[idx][1]
        kpm = self.K_extend[idx][1]
        hpm = self.H_extend[idx][1]
        mpm = self.M_extend[idx][1]
        jpm = self.J_extend[idx][1]
        lupm = self.Lu_extend[idx][1]
        # no pm for Lucknow

#Add Data Augmentation**
        img = transforms.Compose([transforms.ToPILImage(),
                                  transforms.ToTensor()])

        L = img(self.Limg)
        K = img(self.Kimg)
        D = img(self.Dimg)
        H = img(self.Himg)
        M = img(self.Mimg)
        J = img(self.Jimg)
        Lu = img(self.Luimg)



        return L, lpm, D, dpm, K, kpm, H, hpm, M, mpm, J, jpm, Lu, lupm



In [4]:
c = CustomPlaces365Dataset(root=root, transform=transform)

In [5]:
train_size = (int(0.8 * len(c)))
val_size = (len(c) - train_size)

train_set, val_set = torch.utils.data.random_split(c, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=4, shuffle=True)
val_loader = DataLoader(val_set, batch_size=4)

In [6]:
len(val_set)

12498

In [7]:
class Feature_Extractor(nn.Module):
    def __init__(self):
        super(Feature_Extractor, self).__init__()
        resnet50 = models.resnet50(pretrained=True)
        
        
        for param in resnet50.parameters():
            param.requires_grad = False
        
        # Model initialization
        self.resnet_pretrained = resnet50                  #Running images through CNN
        self.fc1 = nn.Linear(self.resnet_pretrained.fc.out_features, 50)
        self.dropout = nn.Dropout(p=0.2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(50, 13)        
    def forward(self, image):
        img_features = self.resnet_pretrained(image)
        img_features = torch.flatten(img_features, 1)
        img_features = self.fc1(img_features)
        x = self.relu(img_features)
        x = self.dropout(x)
        x = self.fc2(x.float())
        return x

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import transforms



# Define the model
model = Feature_Extractor()  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0002)



In [9]:
best_accuracy =0
trigger_times = 0
patience = 50
num_epochs = 300
for epoch in range(num_epochs):
    model.train()  
    train_loss = 0.0
    train_correct = 0

    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()

        train_loss += loss.item() * images.size(0)

    train_loss /= len(train_loader.dataset)
    train_accuracy = train_correct / len(train_loader.dataset)

#     # Validation
    model.eval()  
    val_loss = 0.0
    val_correct = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()

            val_loss += loss.item() * images.size(0)

    val_loss /= len(val_loader.dataset)
    val_accuracy = val_correct / len(val_loader.dataset)
    if val_accuracy > best_accuracy:
        trigger_times = 0
        best_accuracy = val_accuracy
        best_model_weights = model.state_dict()
        torch.save(model.state_dict(), 'best_model_weights.pt')

    else: 
        trigger_times += 1
        print(f"Triggered on epoch {epoch}: {trigger_times}/{patience}")
        if trigger_times >= patience:
            print(f"Early stopping on epoch {epoch} - patience reached")
            break
    print(f'Epoch [{epoch+1}/{num_epochs}] '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f} '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}')

KeyboardInterrupt: 